The goal of this notebook is to obtain number of fragments and variance metric for GFRO decompositions

In [1]:
import sys
sys.path.append('../')

import numpy as np
import pickle

from utils_process import (
    chem_ten2op,
    calculate_variance_metric,
    calculate_variance_metric_memory_efficient
)

from utils_gfro import (
    num_params,
    get_fragment,
    evaluate_cost_function
)


In [2]:
for moltag in ['h2', 'h4', 'lih', 'beh2', 'h2o', 'nh3']:

    # load hamiltonian
    with open(f'../hamiltonians/{moltag}/operator', 'rb') as f:
        Hferm = pickle.load(f)

    # load hamiltonian tensors
    with open(f'../hamiltonians/{moltag}/chem_tensors', 'rb') as f:
        Hobt, Htbt = pickle.load(f)

    # load parameters and norms of gfro decomposition
    with open(f'../results/{moltag}/params_gfro_100', 'rb') as f:
        params, norms = pickle.load(f)

    # convert the parameters to fragment tensors 
    N = Hobt.shape[0]

    tbt_frags = list()
    for x in params:
        tbt_frags.append( get_fragment(x, N) )

    # compute both metrics

    num_fragments_metric = len(tbt_frags) + 1
    variance_metric      = calculate_variance_metric_memory_efficient(Hferm, Hobt, tbt_frags)[1]

    print(f"""
        Molecule            : {moltag}
        Method              : {'gfro'}
        Number of Fragments : {num_fragments_metric}
        Variance Metric     : {variance_metric}
    """)

fragment number: 2 / 3
        Molecule            : h2
        Method              : gfro
        Number of Fragments : 3
        Variance Metric     : (0.13644846968521346-7.530424220218472e-16j)
    
fragment number: 14 / 15
        Molecule            : h4
        Method              : gfro
        Number of Fragments : 15
        Variance Metric     : (0.6571711111928602+2.093081923277021e-17j)
    
fragment number: 23 / 24
        Molecule            : lih
        Method              : gfro
        Number of Fragments : 24
        Variance Metric     : (2.6827082331513434-1.0067901473186767e-15j)
    
fragment number: 31 / 32
        Molecule            : beh2
        Method              : gfro
        Number of Fragments : 32
        Variance Metric     : (1.4712488345255113-1.5019822303346644e-14j)
    
fragment number: 33 / 34
        Molecule            : h2o
        Method              : gfro
        Number of Fragments : 34
        Variance Metric     : (49.715066404683895+

In [ ]:
#
#    sanity check: verify that ||Htbt - \sum_{k=1}^{K} tbt_frags[k]|| == norms[k+1]
#

Htbt_c = Htbt.copy()
current_norm = np.sum(Htbt_c * Htbt_c)
print(np.abs(current_norm - norms[0]) < 1e-12, end='') 

for i, frag in enumerate(tbt_frags):
    Htbt_c -= frag
    current_norm = np.sum(Htbt_c * Htbt_c)
    print(np.abs(current_norm - norms[i+1]) < 1e-12, end='')